<a href="https://colab.research.google.com/github/guilherme-ro/reconhecimento-facial-com-mtcnn-e-facenet/blob/main/reconhecimento_facial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# ----------------------------------------------------------------------
# CÉLULA 1: INSTALAÇÃO E CONFIGURAÇÃO (MTCNN + FaceNet/Keras-Facenet)
# ----------------------------------------------------------------------

# 1. Instalação de bibliotecas essenciais
print("[INFO] Instalando bibliotecas. Por favor, aguarde...")
# Remove qualquer resquício de face_recognition e dlib
!pip uninstall -y face_recognition dlib

# Instala MTCNN e dependências
!pip install mtcnn opencv-python numpy scikit-learn lz4

# Instala a implementação do FaceNet para extração de features
!pip install keras-facenet

import cv2
import numpy as np
import os
import pickle
from google.colab import files
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import matplotlib.pyplot as plt

# Importar as classes necessárias
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet # Novo modelo de extração de features!

# 2. Definição dos caminhos locais e estrutura
DATASET_PATH = "dataset_externo"
ENCODINGS_FILE = "face_encodings.pickle"
MODEL_FILE = "face_recognizer_model.pkl"
TEST_IMAGE_PATH = "data/friends_from_college.webp"

!mkdir -p data
!mkdir -p $DATASET_PATH

# 3. Download da imagem de teste (Placeholder)
print("[INFO] Baixando imagem de teste placeholder...")
!wget -O $TEST_IMAGE_PATH "https://www.publicdomainpictures.net/pictures/320000/velka/group-of-friends-at-a-bar-1574345265o7T.jpg"

print("\n[INFO] Configuração sem Dlib/CUDA concluída. Execute a Célula 2 a seguir.")

[INFO] Instalando bibliotecas. Por favor, aguarde...
Found existing installation: face-recognition 1.3.0
Uninstalling face-recognition-1.3.0:
  Successfully uninstalled face-recognition-1.3.0
Found existing installation: dlib 19.24.6
Uninstalling dlib-19.24.6:
  Successfully uninstalled dlib-19.24.6
  Preparing metadata (setup.py) ... done
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10367 sha256=e75c8a5caa2b63728f5c7c3a960a1d75fff80c1c3198e5a19510c2c0f1ef517d
  Stored in directory: /root/.cache/pip/wheels/05/b0/f5/19ac49fedc10b1df3ee56b096edbcfa39d45794fccc6bcdbbf
Successfully built keras-facenet
[INFO] Baixando imagem de teste placeholder...
--2025-10-04 01:49:24--  https://www.publicdomainpictures.net/pictures/320000/velka/group-of-friends-at-a-bar-1574345265o7T.jpg
Resolving www.publicdomainpictures.net (www.publicdomainpictures.net)... 172.66.134.60, 172.66.140.55, 2606:4700:10::ac42:863c, ...
Connecting to www.publicdomainpictures.net (www

In [9]:
# ----------------------------------------------------------------------
# CÉLULA 2: TREINAMENTO DO MODELO (MTCNN para Detecção, FaceNet para Encoding)
# ----------------------------------------------------------------------

# Inicializa o detector MTCNN e o extrator FaceNet
detector_mtcnn = MTCNN()
# Esta classe também carrega o modelo neural pré-treinado (FaceNet)
encoder_facenet = FaceNet()

def treinar_modelo_facenet(dataset_path, encodings_file, model_file, detector, encoder):
    print("[INFO] Quantificando faces...")

    person_names = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

    if not person_names:
        print("[ERRO FATAL] Nenhuma pasta de pessoa encontrada. Verifique se 'dataset_externo' está populada.")
        return None, None

    knownEncodings = []
    knownNames = []

    for name in person_names:
        person_dir = os.path.join(dataset_path, name)
        print(f"[INFO] Processando {name}...")

        for filename in os.listdir(person_dir):
            if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
                path = os.path.join(person_dir, filename)

                # 1. Carregar Imagem
                image = cv2.imread(path)
                if image is None: continue

                rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                # 2. DETECÇÃO FACIAL (MTCNN)
                results = detector.detect_faces(rgb)

                face_images = []
                for face_data in results:
                    x, y, w, h = face_data['box']
                    # Recorta a face da imagem e redimensiona (FaceNet espera 160x160)
                    face = rgb[y:y+h, x:x+w]
                    face = cv2.resize(face, (160, 160))
                    face_images.append(face)

                if face_images:
                    # 3. EXTRAÇÃO DE FEATURES/EMBEDDINGS (FaceNet)
                    # O FaceNet processa as faces recortadas e retorna os embeddings
                    embeddings = encoder.embeddings(face_images)

                    for embedding in embeddings:
                        knownEncodings.append(embedding)
                        knownNames.append(name)

    print(f"[INFO] Total de {len(knownEncodings)} faces processadas para treinamento.")

    if not knownEncodings:
        print("[ERRO FATAL] Nenhuma face detectada.")
        return None, None

    # 4. Serializar Encodings, Treinar SVM e Salvar
    print("[INFO] Treinando o modelo SVM...")
    le = LabelEncoder()
    labels = le.fit_transform(knownNames)

    recognizer = SVC(kernel="linear", probability=True)
    recognizer.fit(knownEncodings, labels)

    print("[INFO] Salvando o modelo e o encoder...")
    with open(model_file, 'wb') as f:
        pickle.dump((recognizer, le), f)

    print("[INFO] Treinamento concluído. Modelo salvo em:", model_file)
    return recognizer, le

# **EXECUTE O TREINAMENTO**
try:
    recognizer, le = treinar_modelo_facenet(DATASET_PATH, ENCODINGS_FILE, MODEL_FILE, detector_mtcnn, encoder_facenet)
except Exception as e:
    print(f"\n[ERRO INESPERADO] Ocorreu um erro durante o treinamento: {e}")

[INFO] Quantificando faces...
[INFO] Processando annie...
[INFO] Processando billy...
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
[INFO] Processando .ipynb_checkpoints...
[INFO] Processando cobie...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
[INFO] Total de 2 faces processadas para treinamento.
[INFO] Treinando o modelo SVM...
[INFO] Salvando o modelo e o encoder...
[INFO] Treinamento concluído. Modelo salvo em: face_recognizer_model.pkl


In [10]:
# ----------------------------------------------------------------------
# CÉLULA 3: DETECÇÃO, RECONHECIMENTO E VISUALIZAÇÃO (MTCNN + FaceNet)
# ----------------------------------------------------------------------

# Carregar o modelo treinado e re-inicializar o encoder FaceNet (se a sessão tiver sido reiniciada)
try:
    with open(MODEL_FILE, 'rb') as f:
        (recognizer, le) = pickle.load(f)
    print("[INFO] Modelo e Label Encoder carregados com sucesso.")

    # Re-inicializa os detectores necessários
    detector_mtcnn = MTCNN()
    encoder_facenet = FaceNet()
except FileNotFoundError:
    print("[ERRO] Arquivos de modelo não encontrados. Execute a Célula 2 com sucesso.")
    raise SystemExit

def reconhecer_multiplas_faces_facenet(image_path, recognizer, le, detector, encoder):
    print(f"[INFO] Processando imagem: {image_path}...")

    image = cv2.imread(image_path)
    if image is None: return None

    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 1. DETECÇÃO FACIAL (MTCNN)
    print("[INFO] Detectando faces...")
    results = detector.detect_faces(rgb)

    face_images = []
    boxes_coordinates = []

    for face_data in results:
        x, y, w, h = face_data['box']

        # Recorte e Redimensionamento (160x160)
        face = rgb[y:y+h, x:x+w]
        face = cv2.resize(face, (160, 160))
        face_images.append(face)

        # Armazena as coordenadas para desenhar
        boxes_coordinates.append((y, x + w, y + h, x)) # (top, right, bottom, left)

    names = []
    if face_images:
        # 2. EXTRAÇÃO DE FEATURES/EMBEDDINGS (FaceNet)
        embeddings = encoder.embeddings(face_images)

        # 3. CLASSIFICAÇÃO/RECONHECIMENTO (SVM)
        for embedding in embeddings:
            preds = recognizer.predict_proba([embedding])[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = le.classes_[j]

            if proba * 100 > 60:
                name = f"{name}: {proba*100:.1f}%"
            else:
                name = "Desconhecido"

            names.append(name)

    # 4. Desenhar Bounding Boxes e Rótulos
    for ((top, right, bottom, left), name) in zip(boxes_coordinates, names):
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        y_text = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(image, name, (left, y_text), cv2.FONT_HERSHEY_SIMPLEX,
                    0.75, (0, 255, 0), 2)

    print(f"[INFO] {len(boxes_coordinates)} faces detectadas e rotuladas.")
    return image

# **EXECUÇÃO E VISUALIZAÇÃO**
output_image = reconhecer_multiplas_faces_facenet(TEST_IMAGE_PATH, recognizer, le, detector_mtcnn, encoder_facenet)

if output_image is not None:
    print("\n[RESULTADO] Imagem com faces detectadas e reconhecidas:")

    rgb_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(12, 12))
    plt.imshow(rgb_image)
    plt.axis("off")
    plt.show()

    output_path = "resultado_reconhecimento.jpg"
    cv2.imwrite(output_path, output_image)
    print(f"\n[INFO] Imagem de resultado salva como: {output_path}")

[INFO] Modelo e Label Encoder carregados com sucesso.
[INFO] Processando imagem: data/friends_from_college.webp...
